# Bounding Boxes

In this notebook, I will add the bounding boxes from the labels to the images. I will use openCV to draw the bounding boxes.

The txt files have the following format:

```txt
tumor_type x y width height
```

where `tumor_type` is the type of the tumor, `x` and `y` are the coordinates of the top-left corner of the bounding box, and `width` and `height` are the width and height of the bounding box.


In [1]:
import cv2 as cv
from numpy import zeros, array, uint8


def draw_bounding_box(image_path: str, label_path: str, darken_factor: float = 0.6):
  img = cv.imread(image_path)

  # Get the img sizes for scaling the coordinates
  img_height, img_width = img.shape[:2]

  with open(label_path, 'r') as f:
    text = f.readline().strip()

  tumor_type, x, y, w, h = text.split()

  # Set the correct types
  tumor_type = int(tumor_type)
  x, y, w, h = map(float, [x, y, w, h])

  # Scale the float values to image dimensions
  x = int(x * img_width)
  y = int(y * img_height)
  w = int(w * img_width)
  h = int(h * img_height)

  # Get the top left values of x and y
  x = int(x - (w / 2))
  y = int(y - (h / 2))

  point1 = (x, y)
  point2 = (x + w, y + h)

  # Display bounding box
  cv.rectangle(img, point1, point2, (255, 0, 255), 2)

  # Make the rest of the img darker
  mask = zeros(img.shape, uint8)
  cv.rectangle(mask, point1, point2, (255, 255, 255), -1)

  darken_factor = 1 - darken_factor
  inverted_mask = cv.bitwise_not(mask)
  darkened = cv.multiply(img, array([darken_factor, darken_factor, darken_factor]))

  final = cv.bitwise_and(darkened, inverted_mask) + cv.bitwise_and(img, mask)

  # Display the image with the transparent bounding box
  cv.imshow("Mammogram with Bounding Box", final)

  # Wait until the 'Esc' key or the 'X' button on the window is pressed to close
  while True:
    if cv.getWindowProperty("Mammogram with Bounding Box", cv.WND_PROP_VISIBLE) < 1:
      break
    if cv.waitKey(1) == 27:  # 'Esc' key
      break

  cv.destroyAllWindows()

In [2]:
from os import walk


def get_files(path: str):
  files: list[str] = []

  for (dirpath, dirnames, filenames) in walk(path):
    files.extend(filenames)
    break

  return files


def remove_file_extension(file: str) -> str:
  return file.rsplit(".", 1)[0]


def get_files_names(path: str) -> list[str]:
  return [remove_file_extension(file) for file in get_files(path)]

In [4]:
img_path = "./dataset/test/images/"
label_path = "./dataset/test/labels/"

files = get_files_names(img_path)

for file in files:
  img_file = img_path + file + ".jpg"
  label_file = label_path + file + ".txt"

  draw_bounding_box(img_file, label_file)